In [1]:
import pandas_datareader.data as web
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
#plt.style.use('ggplot')
%matplotlib inline

In [2]:
etf_df = pd.read_csv("resources/etf_df.csv")

### Period Datasets
1M, 3M, 6M, 1Y, 5Y, 10Y, and MAX

In [3]:
# last date configured
last_date = list(etf_df.Date)[-1]
last_split = last_date.split('-')
d_list = [int(i) for i in last_split]
date = dt.date(d_list[0], d_list[1], d_list[2])

In [4]:
# calculate the dates from the last data point for 1m, 3m, 6m, 1y, 5y, and 10y...
# ...then convert to string format for filtering DataFrame
one_mo = (date - dt.timedelta(days=30.4167)).strftime("%Y-%m-%d")
three_mo = (date - dt.timedelta(days=91.2501)).strftime("%Y-%m-%d")
six_mo = (date - dt.timedelta(days=182.5)).strftime("%Y-%m-%d")
one_yr = (date - dt.timedelta(days=365)).strftime("%Y-%m-%d")
five_yr = (date - dt.timedelta(days=1825)).strftime("%Y-%m-%d")
ten_yr = (date - dt.timedelta(days=3650)).strftime("%Y-%m-%d")

In [5]:
etf_df.loc[(etf_df.symbol == 'QQQ')&(etf_df.Date >= ten_yr)&(etf_df.Date <= last_date)]

,Date,Open,High,Low,Close,Adj Close,Volume,symbol
3072,2011-05-24,57.060001,57.070000,56.560001,56.570000,51.138706,39314800,QQQ
3073,2011-05-25,56.500000,57.060001,56.470001,56.790001,51.337589,41369000,QQQ
3074,2011-05-26,56.619999,57.299999,56.610001,57.139999,51.653973,40564100,QQQ
3075,2011-05-27,57.240002,57.490002,57.189999,57.430000,51.916138,30127200,QQQ
3076,2011-05-31,57.970001,58.360001,57.669998,58.360001,52.756824,47590100,QQQ
...,...,...,...,...,...,...,...,...
5583,2021-05-17,324.630005,326.369995,321.450012,324.410004,324.410004,39395000,QQQ
5584,2021-05-18,325.390015,326.649994,321.940002,322.220001,322.220001,36528500,QQQ
5585,2021-05-19,316.660004,322.899994,316.299988,322.589996,322.589996,66319100,QQQ
5586,2021-05-20,324.119995,329.709991,323.890015,328.829987,328.829987,46466100,QQQ


### 1. Relative Returns

In [6]:
# three lists created to cover possible portfolio combinations to display
qqq_symbols = ['QQQ', 'SPLG', 'DIA', 'VEU']
sp_symbols = ['QQQ', 'SPLG', 'IVV', 'VOO', 'SPY', 'DIA']
glob_symbols = ['QQQ', 'SPLG', 'ACWI', 'VEU', 'VSS']

#list of lists...
portfolios = [qqq_symbols, sp_symbols, glob_symbols]

# list of years we need a portfolio display for
dates_list = [one_mo, three_mo, six_mo, one_yr, five_yr, ten_yr]
dates_str = ['1M', '3M', '6M', '1Y', '5Y', '10Y']

In [7]:
# create function that exports a DataFrame for the list of symbols' relative returns
def rel_returns(symbols, start):
    df = pd.DataFrame(columns = symbols)
    for symbol in symbols:
        df_filter = etf_df.loc[(etf_df.symbol == symbol)&(etf_df.Date >= start)&(etf_df.Date <= last_date)]
        df[symbol] = list(df_filter.Close)
    df['Date'] = list(df_filter.Date)
    df.set_index("Date", inplace = True)
    returns = (df/df.iloc[0])
    return returns

def close_compare(symbols, start):
    df = pd.DataFrame(columns = symbols)
    for symbol in symbols:
        df_filter = etf_df.loc[(etf_df.symbol == symbol)&(etf_df.Date >= start)&(etf_df.Date <= last_date)]
        df[symbol] = list(df_filter.Close)
    df['Date'] = list(df_filter.Date)
    df.set_index("Date", inplace = True)
    return df

In [8]:
# create function that exports two charts for every period specified in dates_list...
# ...one for Close price and one for relative returns
def returns_charts(symbols):
    for x in range(len(dates_list)):
        new_df = rel_returns(symbols, dates_list[x])
        new_df.plot(figsize=(10,8))
        plt.title(f'Relative Returns: {dates_str[x]}, {last_date}', fontsize=24)
        plt.xlabel('Date', fontsize=22)
        plt.legend(fontsize=18)
        plt.grid()
        plt.tight_layout()
        plt.savefig(f"../images/returns/returns_{symbols[-1]}_{dates_str[x]}.jpg", dpi=75)
        plt.clf()
        
def close_charts(symbols):
    for x in range(len(dates_list)):
        new_df = close_compare(symbols, dates_list[x])
        new_df.plot(figsize=(10,8))
        plt.title(f'Close: {dates_str[x]}, {last_date}', fontsize=24)
        plt.xlabel('Date', fontsize=22)
        plt.legend(fontsize=18)
        plt.grid()
        plt.tight_layout()
        plt.savefig(f"../images/close/close_{symbols[-1]}_{dates_str[x]}.jpg", dpi=75)
        plt.clf()

In [9]:
returns_charts(qqq_symbols)
returns_charts(sp_symbols)
returns_charts(glob_symbols)

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

...

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

In [10]:
close_charts(qqq_symbols)
close_charts(sp_symbols)
close_charts(glob_symbols)

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

...

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

### 2. Volatility - Histograms

In [11]:
df = rel_returns(qqq_symbols, ten_yr)

In [12]:
df

,QQQ,SPLG,DIA,VEU
Date,,,,
2011-05-24,1.000000,1.000000,1.000000,1.000000
2011-05-25,1.003889,0.999197,1.002839,1.005578
2011-05-26,1.010076,0.999197,1.004055,1.014463
2011-05-27,1.015202,1.008032,1.007543,1.022107
2011-05-31,1.031642,1.014137,1.017843,1.040496
...,...,...,...,...
2021-05-17,5.734665,3.138956,2.787105,1.295041
2021-05-18,5.695952,3.113253,2.766504,1.302066
2021-05-19,5.702492,3.104900,2.752960,1.292769


In [13]:
# create histograms for returns, qqq_symbols list
for x in range(len(dates_list)):
    df = rel_returns(qqq_symbols, dates_list[x])
    df.QQQ.hist(bins=50, figsize=(10,8), label='QQQ', alpha=0.5)
    df.SPLG.hist(bins=50, label='SPLG', alpha=0.5)
    df.DIA.hist(bins=50, label='DIA', alpha=0.5)
    df.VEU.hist(bins=50, label='VEU', alpha=0.5)
    plt.title(f'Volatility: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.savefig(f"../images/vol_hist/histogram_qqq_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()

<Figure size 720x576 with 0 Axes>

In [14]:
# create histograms kernel for clarity, qqq_symbols list
for x in range(len(dates_list)):
    df = rel_returns(qqq_symbols, dates_list[x])
    df.QQQ.plot(kind='kde', label='QQQ', figsize=(10,8))
    df.SPLG.plot(kind='kde', label='SPLG')
    df.DIA.plot(kind='kde', label='DIA')
    df.VEU.plot(kind='kde', label='VEU')
    plt.title(f'Volatility Normalized: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.grid()
    plt.savefig(f"../images/vol_hist/ked_hist_qqq_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()

<Figure size 720x576 with 0 Axes>

In [15]:
# repeat operations for sp_symbols, glob_symbols
for x in range(len(dates_list)):
    df = rel_returns(sp_symbols, dates_list[x])
    df.QQQ.hist(bins=50, figsize=(10,8), label='QQQ', alpha=0.5)
    df.SPLG.hist(bins=50, label='SPLG', alpha=0.5)
    df.IVV.hist(bins=50, label='IVV', alpha=0.5)
    df.VOO.hist(bins=50, label='VOO', alpha=0.5)
    df.SPY.hist(bins=50, label='SPY', alpha=0.5)
    df.DIA.hist(bins=50, label='DIA', alpha=0.5)
    plt.title(f'Volatility: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.savefig(f"../images/vol_hist/histogram_sp_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()
for x in range(len(dates_list)):
    df = rel_returns(sp_symbols, dates_list[x])
    df.QQQ.plot(kind='kde', label='QQQ', figsize=(10,8))
    df.SPLG.plot(kind='kde', label='SPLG')
    df.IVV.plot(kind='kde', label='IVV')
    df.VOO.plot(kind='kde', label='VOO')
    df.SPY.plot(kind='kde', label='SPY')
    df.DIA.plot(kind='kde', label='DIA')
    plt.title(f'Volatility Normalized: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.grid()
    plt.savefig(f"../images/vol_hist/ked_hist_sp_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()

<Figure size 720x576 with 0 Axes>

In [16]:
for x in range(len(dates_list)):
    df = rel_returns(glob_symbols, dates_list[x])
    df.QQQ.hist(bins=50, figsize=(10,8), label='QQQ', alpha=0.5)
    df.SPLG.hist(bins=50, label='SPLG', alpha=0.5)
    df.ACWI.hist(bins=50, label='ACWI', alpha=0.5)
    df.VEU.hist(bins=50, label='VEU', alpha=0.5)
    df.VSS.hist(bins=50, label='VSS', alpha=0.5)
    plt.title(f'Volatility: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.savefig(f"../images/vol_hist/histogram_glob_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()
for x in range(len(dates_list)):
    df = rel_returns(glob_symbols, dates_list[x])
    df.QQQ.plot(kind='kde', label='QQQ', figsize=(10,8))
    df.SPLG.plot(kind='kde', label='SPLG')
    df.ACWI.plot(kind='kde', label='ACWI')
    df.VEU.plot(kind='kde', label='VEU')
    df.VSS.plot(kind='kde', label='VSS')
    plt.title(f'Volatility Normalized: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.grid()
    plt.savefig(f"../images/vol_hist/ked_hist_glob_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()

<Figure size 720x576 with 0 Axes>

### Correlation

In [17]:
from pandas.plotting import scatter_matrix

In [18]:
def scatter_plots(portfolio_list):
    for portfolio in portfolio_list:
        for x in range(len(dates_list)):
            df = close_compare(portfolio, dates_list[x])
            scatter_matrix(df, figsize=(10,10), color='dodgerblue', hist_kwds={'color':'lightblue', 'alpha':0.50, 'bins':50})
            plt.tight_layout()
            plt.savefig(f"../images/correlation/scatter_{portfolio[-1]}_{dates_str[x]}.jpg", dpi=150)
            plt.clf()

In [19]:
scatter_plots(portfolios)

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

### Monte Carlo Simulation
Currently discontinued

In [20]:
# create function that randomly selects a 60 day period 1000 times, plots the simulation and...
# an easier to read quantile version
#def monte_carlo(portfolio):
    #for x in range(len(dates_list)):
        #stocks = close_compare(portfolio, dates_list[x])
        #prf_returns = (stocks.pct_change() +1)[1:]
        #simulated = pd.DataFrame([((prf_returns.iloc[random.choices(range(len(prf_returns)), k=60)])\
                        #.mean(axis=1)).cumprod().values for x in range(1000)]).T
        #simulated.plot(legend=False, linewidth=1, alpha=0.1, color='blue', figsize=(12,8))
        #plt.tight_layout()
        #plt.savefig(f"../images/monte_carlo/simulated_{portfolio[-1]}_{dates_str[x]}.jpg", dpi=150)
        #plt.clf()
        #simulated.quantile([0.05, 0.50, 0.95], axis=1).T.plot(figsize=(12,8))
        #plt.legend(fontsize=18)
        #plt.tight_layout()
        #plt.savefig(f"../images/monte_carlo/quantile_{portfolio[-1]}_{dates_str[x]}.jpg", dpi=75)
        #plt.clf()

In [21]:
#monte_carlo(qqq_symbols)

In [22]:
#monte_carlo(sp_symbols)

In [23]:
#monte_carlo(glob_symbols)